### Import Libraries

In [ ]:
import numpy as np
import pandas as pd

import random

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
%matplotlib widget

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display

### Setting the random state

In [ ]:
rs = min(332078,)

In [ ]:
np.random.seed(rs)

## Loading and Preparing the Data

In [ ]:
file_name = 'cla4lsp_customers.csv'
df_tot = pd.read_csv(file_name, sep='\t')

In [ ]:
n = int(df_tot.shape[0] * (2/3))
df_sample = df_tot.sample(n)

In [ ]:
workdf = df_sample.copy()

In [ ]:
labels = ['NumDealsPurchases','AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response','Complain','Recency']
features = ['Year_Birth','Education','Marital_Status','Income','Kidhome','Teenhome','Dt_Customer','MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds','NumWebPurchases','NumCatalogPurchases','NumStorePurchases','NumWebVisitsMonth']
other_columns = ['ID','Z_CostContact','Z_Revenue']

In [ ]:
workdf = workdf.drop(other_columns, axis=1)

In [ ]:
workdf[labels]

In [ ]:
print('\t\tMAX\n\n',workdf[features].max())
print('\n\t\tMIN\n\n',workdf[features].min())

In [ ]:
purchasing_habits = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']

In [ ]:
# Removing random column
random_pick = random.choice(purchasing_habits)
workdf = workdf.drop(random_pick, axis=1)
features.remove(random_pick)

In [ ]:
df_tot.isnull().sum()[lambda x: x>0]

In [ ]:
df_tot.shape

In [ ]:
workdf.isnull().sum()[lambda x: x>0]

In [ ]:
# rows with NaN Income
nan_mask = workdf['Income'].isna()
workdf[nan_mask]

In [ ]:
grouped_df[('Graduation','Married')]

In [ ]:
grouped_df = workdf.groupby(['Education','Marital_Status'])['Income'].mean()

# expected output
exp_out = []
for i in range(len(workdf[nan_mask])):
    exp_out.append(grouped_df[( workdf[nan_mask]['Education'].iloc[i] , workdf[nan_mask]['Marital_Status'].iloc[i] )])
exp_out

In [ ]:
id = [workdf[nan_mask].index][0]

In [ ]:
# Fill the nan values
for i in workdf.index:
    if workdf['Income'].isna:
        workdf.loc[i,'Income'] = grouped_df[( workdf.loc[i,'Education'] , workdf.loc[i,'Marital_Status'] )]

In [ ]:
# Check whether the newly substituted numbers are equal to the expected numbers
workdf.loc[id,'Income'] == exp_out

## Encoding of Categorical Data
- Cambiare i livelli di educazione con valori che vanno da 0 a 4
- marital status: single (0), together (1)
- cambiare anno di nascita con età
- Dt_customer drop

In [ ]:
workdf[features]

In [ ]:
education_array = workdf['Education'].unique()

In [ ]:
m_status_array = workdf['Marital_Status'].unique()

In [ ]:
# Feature encoding for Education and Marital_status (one-hot-encoding)
def one_hot_encode(df,feature,array):
    for elem in array:
        df[elem] = 0
        i = df[df[feature].values == elem].index
        df.loc[i,elem] = 1
        
one_hot_encode(workdf,'Education',education_array)
one_hot_encode(workdf,'Marital_Status',m_status_array)

In [ ]:
# Feature encoding for date (dd, mm, yyyy)

# Split the Dt_Customer column in three columns ['Dt_day', 'Dt_month', 'Dt_year']
workdf[['Dt_day', 'Dt_month', 'Dt_year']] = list(workdf['Dt_Customer'].str.split('-'))

#type(workdf['Dt_day'].iloc[0]) --> str
# str --> int
workdf[['Dt_day', 'Dt_month', 'Dt_year']] = workdf[['Dt_day', 'Dt_month', 'Dt_year']].astype(int)


In [ ]:
# drop Education and marital status columns
workdf = workdf.drop(['Education','Marital_Status','Dt_Customer'], axis=1)

In [ ]:
#Adding new features
features.extend(education_array)
features.extend(m_status_array)
features.extend(['Dt_day', 'Dt_month', 'Dt_year'])

#Remove previus features
features = [elem for elem in features if elem not in ['Education','Marital_Status','Dt_Customer']]

In [ ]:
workdf.iloc[:20, 22:]

In [ ]:
Xworkdf = workdf.copy()

In [ ]:
Xworkdf

## Preprocessing and full-PCA

- Va fatto su tutte le colonne o solo sulle features (escludendo le labels)?

In [ ]:
# create Xworkdf_std and Xworksf_mm using a StandardScaler and a MinMaxScaler (min “ 0, max “ 1)
from sklearn.preprocessing import StandardScaler, MinMaxScaler

Xworkdf_std = StandardScaler()
Xworkdf_std.fit(Xworkdf[features].values)
t_Xworkdf_std = Xworkdf_std.transform(Xworkdf[features].values)

Xworksf_mm = MinMaxScaler()
Xworksf_mm.fit(Xworkdf[features].values)
t_Xworksf_mm = Xworksf_mm.transform(Xworkdf[features].values)

- analyze and comment a comparison of the variances of Xworkdf with the variances of Xworkdf std and Xworkdf mm. What do you observe from this analysis?


In [ ]:
# CONSIDER ONLY THE FEATURES AND NOT THE LABEL FOR PCA

In [ ]:
variance1 = [np.var(Xworkdf[features].iloc[:,i]) for i in range(Xworkdf[features].shape[1])]
variance2 = [np.var(pd.DataFrame(t_Xworkdf_std).iloc[:,i]) for i in range(t_Xworkdf_std.shape[1])]
variance3 = [np.var(pd.DataFrame(t_Xworksf_mm).iloc[:,i]) for i in range(t_Xworksf_mm.shape[1])]

variance1 = np.var(Xworkdf[features].values)
variance2 = np.var(t_Xworkdf_std)
variance3 = np.var(t_Xworksf_mm)

max1 = np.max(Xworkdf[features])
max2 = np.max(t_Xworkdf_std)
max3 = np.max(t_Xworksf_mm)

min1 = np.min(Xworkdf[features])
min2 = np.min(t_Xworkdf_std)
min3 = np.min(t_Xworksf_mm)

mean1 = np.mean(Xworkdf[features])
mean2 = np.mean(t_Xworkdf_std)
mean3 = np.mean(t_Xworksf_mm)
# Print variances
print(f'\nVariance of each feature of non rescaled data:\n {variance1}')
print(f'\nVariance of each feature of standar-scaled data:\n {variance2}')
print(f'\nVariance of each feature of MinMax-scaled data:\n {variance3}')

# Print Max
print(f'\nMax of non rescaled data:\n {max1}')
print(f'\nMax of standar-scaled data:\n {max2}')
print(f'\nMax of MinMax-scaled data:\n {max3}')

# Print Min
print(f'\nMin of non rescaled data:\n {min1}')
print(f'\nMin of standar-scaled data:\n {min2}')
print(f'\nMin of MinMax-scaled data:\n {min3}')

# Print Mean
print(f'\nMean of non rescaled data:\n {mean1}')
print(f'\nMean of standar-scaled data:\n {mean2}')
print(f'\nMean of MinMax-scaled data:\n {mean3}')

- Apply the “full” PCA1 to the DFs Xworkdf, Xworkdf std, and Xworkdf mm and plot the curve of the cumulative explained variance. Looking at the results, improve the analysis and comments made at the previous step

In [ ]:
#Apply the “full” PCA1 to the DFs Xworkdf, Xworkdf_std, and Xworkdf_mm
pca_Xworkdf = PCA()
pca_Xworkdf_std = PCA()
pca_Xworkdf_mm = PCA()

pca_Xworkdf.fit(Xworkdf[features].values) 
pca_Xworkdf_std.fit(t_Xworkdf_std)
pca_Xworkdf_mm.fit(t_Xworksf_mm)

#plot the curve of the cumulative explained variance
plt.figure(figsize=(10,5))
plt.plot(np.cumsum(pca_Xworkdf.explained_variance_ratio_))
plt.title('Xworkdf')
plt.ylim([0, 1.1])
plt.xticks(ticks=np.arange(pca_Xworkdf.n_features_), 
           labels=[f'PC{i + 1}' for i in range(pca_Xworkdf.n_features_)])
plt.xlabel('Principal components')
plt.ylabel('Cumulative explained variance')
plt.grid()
plt.show()

plt.figure(figsize=(10,5))
plt.plot(np.cumsum(pca_Xworkdf_std.explained_variance_ratio_))
plt.title('Xworkdf_std')
plt.ylim([0, 1.1])
plt.xticks(ticks=np.arange(pca_Xworkdf_std.n_features_), 
           labels=[f'PC{i + 1}' for i in range(pca_Xworkdf_std.n_features_)])
plt.xlabel('Principal components')
plt.ylabel('Cumulative explained variance')
plt.grid()
plt.show()

plt.figure(figsize=(10,5))
plt.plot(np.cumsum(pca_Xworkdf_mm.explained_variance_ratio_))
plt.title('Xworkdf_mm')
plt.ylim([0, 1.1])
plt.xticks(ticks=np.arange(pca_Xworkdf_mm.n_features_), 
           labels=[f'PC{i + 1}' for i in range(pca_Xworkdf_mm.n_features_)])
plt.xlabel('Principal components')
plt.ylabel('Cumulative explained variance')
plt.grid()
plt.show()


## Dimensionality Reduction and Interpretation of the PCs

In [ ]:
# Selecting m PCs such that m = min{m', 5}
# m' is the minimum number of PCs that explains 33% of the total variance

m_star_std = 3 # m' of Xworkdf_std
m_star_mm = 3 # m' of Xworkdf_mm

m_std = min(m_star_std,5)
m_mm = min(m_star_mm,5)

In [ ]:
pca_std = PCA(n_components = m_std)
pca_mm = PCA(n_components = m_mm)

# FIT THE PCA
pca_std.fit(t_Xworkdf_std)

# COMPUTE THE PERCENTAGE OF TOT. EXPL. VARIANCE (ROUNDED TO 2 DECIMALS)
round_expl_var_ratio_std = np.round(pca_std.explained_variance_ratio_.sum()*100,2)

# MAKE THE BARPLOT
plt.figure(figsize=(6, 6))
plt.bar(range(1,m_std+1), pca_std.explained_variance_ratio_)
plt.title(f"PCs' EXPLAINED VARIANCE ({round_expl_var_ratio_std}% OF TOT. EXPL. VAR.)")
plt.xticks(ticks=np.arange(1,m_std+1), 
           labels=[f'PC{i}' for i in range(1,m_std+1)],
           rotation=45)
plt.xlabel('Principal Components')
plt.ylabel('Percentage of Explained variance')
plt.grid()
plt.show()

# FIT THE PCA
pca_mm.fit(t_Xworksf_mm)

# COMPUTE THE PERCENTAGE OF TOT. EXPL. VARIANCE (ROUNDED TO 2 DECIMALS)
round_expl_var_ratio_mm = np.round(pca_mm.explained_variance_ratio_.sum()*100,2)

# MAKE THE BARPLOT
plt.figure(figsize=(6, 6))
plt.bar(range(1,m_mm+1), pca_mm.explained_variance_ratio_)
plt.title(f"PCs' EXPLAINED VARIANCE ({round_expl_var_ratio_mm}% OF TOT. EXPL. VAR.)")
plt.xticks(ticks=np.arange(1,m_mm+1), 
           labels=[f'PC{i}' for i in range(1,m_mm+1)],
           rotation=45)
plt.xlabel('Principal Components')
plt.ylabel('Percentage of Explained variance')
plt.grid()
plt.show()

- Given the PCs of Xworkdf std and Xworkdf mm, give them an interpretation and, therefore, a name. Tables and/or plots are welcome;

In [ ]:
# DEFINE EPSILON
eps = np.sqrt(1 / pca_std.n_features_)

# DEFINE THE LIST OF SKILL COLORS W.R.T. THE SKILL TYPES AND THE SKILL CATEGORIES
#skill_colors_type = [skill_types_df.loc[skill_types_df['skill'] == s]['color'].values[0] for s in skill_cols]
#skill_colors_cat = [skill_cats_df.loc[skill_cats_df['skill'] == s]['color'].values[0] for s in skill_cols]

# MAKE A CUSTOM LEGEND
#type_colors_legend = [Line2D([0], [0], color=type_colors[k]) for k in type_colors.keys()]
#cat_colors_legend = [Line2D([0], [0], color=cat_colors[k]) for k in cat_colors.keys()]

# FOR-CYCLE TO GENERALIZE THE PLOT COMMANDS
for ii in range(m_std):
    # MAKE THE BARPLOT WITH SKILL TYPE COLORS
    plt.figure(figsize=(12, 6))
    plt.bar(np.arange(pca_std.n_features_), pca_std.components_[ii, :])#,color=skill_colors_type)
    # --- RED LINE DENOTING THE THRESHOLD [-eps, +eps] -----------------
    plt.plot([-0.5, pca_std.n_features_ - 0.5], [eps, eps], 'red')
    plt.plot([-0.5, pca_std.n_features_ - 0.5], [-eps, -eps], 'red')
    # ------------------------------------------------------------------
    plt.xticks(ticks=np.arange(pca_std.n_features_),
               labels= features,
               rotation=75)
    plt.title(f'SKILLS (COLORED BY TYPE) - PC{ii + 1}')
    #plt.legend(type_colors_legend, [k for k in type_colors.keys()])
    plt.grid()
    plt.tight_layout()
    plt.show()
    
    # THE SELECTION OF THE SKILLS WITH CONTRIBUTE GREATER THAN THE THRESHOLD
    ind_great_pos_PCii = np.argwhere(pca_std.components_[ii, :] >= eps).flatten()
    ind_great_neg_PCii = np.argwhere(pca_std.components_[ii, :] <= -eps).flatten()
    
    great_pos_PCii = [features[i] for i in ind_great_pos_PCii]
    great_neg_PCii = [features[i] for i in ind_great_neg_PCii]
    
    print('')
    print(f'****************** PC{ii+1} **********************')
    print(f'HIGH-VALUED POSITIVE COMPONENTS: {great_pos_PCii}')
    print('')
    print(f'HIGH-VALUED NEGATIVE COMPONENTS: {great_neg_PCii}')
    print('*********************************************')
    print('')

In [ ]:
# DEFINE EPSILON
eps = np.sqrt(1 / pca_mm.n_features_)

# DEFINE THE LIST OF SKILL COLORS W.R.T. THE SKILL TYPES AND THE SKILL CATEGORIES
#skill_colors_type = [skill_types_df.loc[skill_types_df['skill'] == s]['color'].values[0] for s in skill_cols]
#skill_colors_cat = [skill_cats_df.loc[skill_cats_df['skill'] == s]['color'].values[0] for s in skill_cols]

# MAKE A CUSTOM LEGEND
#type_colors_legend = [Line2D([0], [0], color=type_colors[k]) for k in type_colors.keys()]
#cat_colors_legend = [Line2D([0], [0], color=cat_colors[k]) for k in cat_colors.keys()]

# FOR-CYCLE TO GENERALIZE THE PLOT COMMANDS
for ii in range(m_mm):
    # MAKE THE BARPLOT WITH SKILL TYPE COLORS
    plt.figure(figsize=(12, 6))
    plt.bar(np.arange(pca_mm.n_features_), pca_mm.components_[ii, :])#,color=skill_colors_type)
    # --- RED LINE DENOTING THE THRESHOLD [-eps, +eps] -----------------
    plt.plot([-0.5, pca_mm.n_features_ - 0.5], [eps, eps], 'red')
    plt.plot([-0.5, pca_mm.n_features_ - 0.5], [-eps, -eps], 'red')
    # ------------------------------------------------------------------
    plt.xticks(ticks=np.arange(pca_mm.n_features_),
               labels= features,
               rotation=75)
    plt.title(f'SKILLS (COLORED BY TYPE) - PC{ii + 1}')
    #plt.legend(type_colors_legend, [k for k in type_colors.keys()])
    plt.grid()
    plt.tight_layout()
    plt.show()
    
    # THE SELECTION OF THE SKILLS WITH CONTRIBUTE GREATER THAN THE THRESHOLD
    ind_great_pos_PCii = np.argwhere(pca_mm.components_[ii, :] >= eps).flatten()
    ind_great_neg_PCii = np.argwhere(pca_mm.components_[ii, :] <= -eps).flatten()
    
    great_pos_PCii = [features[i] for i in ind_great_pos_PCii]
    great_neg_PCii = [features[i] for i in ind_great_neg_PCii]
    
    print('')
    print(f'****************** PC{ii+1} **********************')
    print(f'HIGH-VALUED POSITIVE COMPONENTS: {great_pos_PCii}')
    print('')
    print(f'HIGH-VALUED NEGATIVE COMPONENTS: {great_neg_PCii}')
    print('*********************************************')
    print('')

- After the interpretation, for both the DFs represent a score graph with respect to the firstlPCs,wherel“2ifm“2andl“3ifmě3. Inparticular,writethenamesof the PCs (chosen by you) on the axes of the plots;

In [ ]:
pc_names_std = ['Customer Purchases (+) vs Monthly Views & Kids (-)',
                'Teenager (+) vs Adult (-)', # Birth & Basic & Single (+) vs Income & Teen & Phd (+)
                'Enroll year (+) vs Enroll month & WebActive & Basic (-)',
                'PhD & Master & Basic (+) vs Graduation (-)'
                ]

In [ ]:
pc_names_mm = ['PhD & Master (+) vs Graduation (-)',
               'Married (+) & Together & Single (-)',
               'Single (+) & Together(-)'
               ]

In [ ]:
#S STD core graph 
Yworkdf_std = pca_std.transform(t_Xworkdf_std)

# MAKE THE 3D SCORE GRAPH
sg_3d = plt.figure(figsize=(8, 8))
ax_sg_3d = sg_3d.add_subplot(111, projection='3d')
ax_sg_3d.scatter(Yworkdf_std[:, 0], Yworkdf_std[:, 1], Yworkdf_std[:, 2], s=2, alpha=0.25)
plt.title('...')
ax_sg_3d.set_xlabel(pc_names_std[0])
ax_sg_3d.set_ylabel(pc_names_std[1])
ax_sg_3d.set_zlabel(pc_names_std[2])
#plt.legend(genpos_colors_legend, [k for k in genpos_colors.keys()])
plt.grid()
plt.show()

# MAKE THE 2D SCORE GRAPH
plt.figure()
plt.scatter(Yworkdf_std[:, 0], Yworkdf_std[:, 1], s=2, alpha=0.25)
plt.title('...')
plt.xlabel(pc_names_std[0])
plt.ylabel(pc_names_std[1])
#plt.legend(genpos_colors_legend, [k for k in genpos_colors.keys()])
plt.grid()
plt.show()

In [ ]:
#S STD core graph 
Yworkdf_mm = pca_mm.transform(t_Xworksf_mm)

# MAKE THE 3D SCORE GRAPH
sg_3d = plt.figure(figsize=(8, 8))
ax_sg_3d = sg_3d.add_subplot(111, projection='3d')
ax_sg_3d.scatter(Yworkdf_mm[:, 0], Yworkdf_mm[:, 1], Yworkdf_mm[:, 2], s=2, alpha=0.25)
plt.title('...')
ax_sg_3d.set_xlabel(pc_names_mm[0])
ax_sg_3d.set_ylabel(pc_names_mm[1])
ax_sg_3d.set_zlabel(pc_names_mm[2])
#plt.legend(genpos_colors_legend, [k for k in genpos_colors.keys()])
plt.grid()
plt.show()

# MAKE THE 2D SCORE GRAPH
plt.figure()
plt.scatter(Yworkdf_mm[:, 0], Yworkdf_mm[:, 1], s=2, alpha=0.25)
plt.title('...')
plt.xlabel(pc_names_mm[0])
plt.ylabel(pc_names_mm[1])
#plt.legend(genpos_colors_legend, [k for k in genpos_colors.keys()])
plt.grid()
plt.show()

- Optional: make more than one score graph, coloring the dots with respect to any label you consider meaningful.

- analyze and comment the results

## K-Means
- Run the k-Means algorithm on the two DFs, with respect to the “PC-space”. Select the best value of k in {3,...,10} using the silhouette coefficient.

In [ ]:
km_list_std = []
silcoeff_list_std = []
k_list_std = list(range(3, 11))

# START THE FOR-CYCLE TO RUN THE k-MEANS AND MEASURING THE SILHOUETTE COEFFICIENT
for i in range(len(k_list_std)):
    print(f'****************** START k-MEANS WITH k={k_list_std[i]} ******************')
    print('Computing...')
    km_list_std.append(KMeans(n_clusters=k_list_std[i], n_init=3, random_state=rs))
    km_std = km_list_std[i]
    km_std.fit(Yworkdf_std)
    silcoeff_list_std.append(silhouette_score(Yworkdf_std,km_std.labels_))
    print(f'****************** END k-MEANS WITH k={k_list_std[i]} ******************')
    print('')

# FIND THE BEST VALUE OF k AND THE BEST KMeans OBJECT
i_best_std = np.argmax(silcoeff_list_std)
k_std = k_list_std[i_best_std]
km_std = km_list_std[i_best_std]

# VISUALIZE THE RESULT
print('')
print('')
print('****************** RESULTS OF THE SEARCH... ******************')
print(f'BEST SILHOUETTE SCORE: {np.max(silcoeff_list_std)} --> k = {k_std}') 
print('**************************************************************')

In [ ]:
km_list_mm = []
silcoeff_list_mm = []
k_list_mm = list(range(3, 11))

# START THE FOR-CYCLE TO RUN THE k-MEANS AND MEASURING THE SILHOUETTE COEFFICIENT
for i in range(len(k_list_mm)):
    print(f'****************** START k-MEANS WITH k={k_list_mm[i]} ******************')
    print('Computing...')
    km_list_mm.append(KMeans(n_clusters=k_list_mm[i], n_init=3, random_state=rs))
    km_mm = km_list_mm[i]
    km_mm.fit(Yworkdf_mm)
    silcoeff_list_mm.append(silhouette_score(Yworkdf_mm,km_mm.labels_))
    print(f'****************** END k-MEANS WITH k={k_list_mm[i]} ******************')
    print('')

# FIND THE BEST VALUE OF k AND THE BEST KMeans OBJECT
i_best_mm = np.argmax(silcoeff_list_mm)
k_mm = k_list_mm[i_best_std]
km_mm = km_list_mm[i_best_std]

# VISUALIZE THE RESULT
print('')
print('')
print('****************** RESULTS OF THE SEARCH... ******************')
print(f'BEST SILHOUETTE SCORE: {np.max(silcoeff_list_mm)} --> k = {k_mm}') 
print('**************************************************************')

## Clusters and Centroid Interpretation and Visualization
- Comment the centroids of the best clustering for both the DFs
  - give to each centroid a name or a meaningful brief description that characterizes the average customer in the cluster
    represented by the centroid.
- plot the score graph of exercise 4 together with the centroids (showing the different clusters using different colors and/or markers for the dots)

In [ ]:
# MAKE THE 3D SCORE GRAPH WITH THE CENTROIDS
sg_3d_km = plt.figure(figsize=(8, 8))
ax_sg_3d_km = sg_3d_km.add_subplot(111, projection='3d')
ax_sg_3d_km.scatter(Yworkdf_std[:, 0], Yworkdf_std[:, 1], Yworkdf_std[:, 2], s=2,alpha=0.15)
ax_sg_3d_km.scatter(km_std.cluster_centers_[:,0], km_std.cluster_centers_[:,1], km_std.cluster_centers_[:,2], c='black')
for kk in range(k_std):
    ax_sg_3d_km.text(km_std.cluster_centers_[kk, 0], km_std.cluster_centers_[kk, 1], km_std.cluster_centers_[kk, 2], f'clust.{kk+1}')
plt.title('...')
ax_sg_3d_km.set_xlabel(pc_names_std[0])
ax_sg_3d_km.set_ylabel(pc_names_std[1])
ax_sg_3d_km.set_zlabel(pc_names_std[2])
#plt.legend(genpos_colors_legend, [k for k in genpos_colors.keys()])
plt.grid()
plt.show()

In [ ]:
# MAKE THE 3D SCORE GRAPH WITH THE CENTROIDS
sg_3d_km = plt.figure(figsize=(8, 8))
ax_sg_3d_km = sg_3d_km.add_subplot(111, projection='3d')
ax_sg_3d_km.scatter(Yworkdf_mm[:, 0], Yworkdf_mm[:, 1], Yworkdf_mm[:, 2], s=2,alpha=0.05)
ax_sg_3d_km.scatter(km_mm.cluster_centers_[:,0], km_mm.cluster_centers_[:,1], km_mm.cluster_centers_[:,2], c='black')
for kk in range(k_mm):
    ax_sg_3d_km.text(km_mm.cluster_centers_[kk, 0], km_mm.cluster_centers_[kk, 1], km_mm.cluster_centers_[kk, 2], f'clust.{kk+1}')
plt.title('...')
ax_sg_3d_km.set_xlabel(pc_names_mm[0])
ax_sg_3d_km.set_ylabel(pc_names_mm[1])
ax_sg_3d_km.set_zlabel(pc_names_mm[2])
#plt.legend(genpos_colors_legend, [k for k in genpos_colors.keys()])
plt.grid()
plt.show()

In [ ]:
# COMPUTE THE MAX/MIN VALUES IN THE PC-SPACE
maxs_y = Yworkdf_std.max(axis=0) 
mins_y = Yworkdf_std.min(axis=0) 

# MAKE THE BARPLOTS OF THE CENTROIDS
fig_centroids, ax_centroids = plt.subplots(2, 2, figsize=(10, 15))
for ii in range(k_std):
    ir = ii // 2
    ic = ii % 2
    ax_centroids[ir, ic].bar(np.arange(km_std.cluster_centers_.shape[1]), maxs_y, color='blue', alpha=0.15)
    ax_centroids[ir, ic].bar(np.arange(km_std.cluster_centers_.shape[1]), mins_y, color='blue', alpha=0.15)
    ax_centroids[ir, ic].bar(np.arange(km_std.cluster_centers_.shape[1]), km_std.cluster_centers_[ii,:])
    ax_centroids[ir, ic].set_xticks(ticks=np.arange(km_std.cluster_centers_.shape[1]))
    ax_centroids[ir, ic].set_xticklabels(labels=pc_names_std, rotation=75)
    ax_centroids[ir, ic].grid(visible=True, which='both')
    plt.tight_layout()
    ax_centroids[ir, ic].set_title(f'CENTROID {ii+1}')

In [ ]:
# COMPUTE THE MAX/MIN VALUES IN THE PC-SPACE
maxs_y = Yworkdf_mm.max(axis=0) 
mins_y = Yworkdf_mm.min(axis=0) 

# MAKE THE BARPLOTS OF THE CENTROIDS
fig_centroids, ax_centroids = plt.subplots(2, 2, figsize=(10, 15))
for ii in range(k_mm):
    ir = ii // 2
    ic = ii % 2
    ax_centroids[ir, ic].bar(np.arange(km_mm.cluster_centers_.shape[1]), maxs_y, color='blue', alpha=0.15)
    ax_centroids[ir, ic].bar(np.arange(km_mm.cluster_centers_.shape[1]), mins_y, color='blue', alpha=0.15)
    ax_centroids[ir, ic].bar(np.arange(km_mm.cluster_centers_.shape[1]), km_mm.cluster_centers_[ii,:])
    ax_centroids[ir, ic].set_xticks(ticks=np.arange(km_mm.cluster_centers_.shape[1]))
    ax_centroids[ir, ic].set_xticklabels(labels=pc_names_mm, rotation=75)
    ax_centroids[ir, ic].grid(visible=True, which='both')
    plt.tight_layout()
    ax_centroids[ir, ic].set_title(f'CENTROID {ii+1}')

In [ ]:
cluster_names_std = ['1',
                     '2',
                     '3',
                     '4'
                     ]

In [ ]:
cluster_names_mm = ['1',
                     '2',
                     '3',
                     '4'
                     ]

## Clusters and Centroid Evaluation
- For both the DFs, perform an internal and an external evaluation of the clusterings obtained
  - Measure the silhouette scores of the clusters (internal evaluation);
  - perform an external evaluation of the clusters analyzing and plotting the distribution of the labels (view lesson 22/12/23)
  - Comment the results. Compare the results obtained from Xworkdf std and Xworkdf mm and comment them

In [ ]:
silscores = silhouette_samples(Yworkdf_std, km_std.labels_)
cluster_silscores = [np.mean (silscores [km_std.labels_ == kk]) for kk in range (k_std) ]
display(pd.DataFrame(np.array(cluster_silscores + [np.max(silcoeff_list_std)]), index=cluster_names_std + ['Global'], columns=['Sil. Score' ]))

In [ ]:
silscores = silhouette_samples(Yworkdf_mm, km_mm.labels_)
cluster_silscores = [np.mean (silscores [km_mm.labels_ == kk]) for kk in range (k_mm) ]
display(pd.DataFrame(np.array(cluster_silscores + [np.max(silcoeff_list_mm)]), index=cluster_names_mm + ['Global'], columns=['Sil. Score' ]))

In [ ]:
#EXTERNAL EVALUATION

In [ ]:
Xworkdf.columns